In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

base_model = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(base_model)

tokenizer = AutoTokenizer.from_pretrained(base_model, device_map="auto")
model = AutoModelForSequenceClassification.from_pretrained(base_model, load_in_8bit=True, torch_dtype=torch.float16)

model = model.eval()

In [2]:
tokens = tokenizer(["Fuck this economy. I hate aig and their non loan given asses"], return_tensors='pt',
                           padding=True)
output = model(**tokens)

output = torch.nn.functional.softmax(output.logits.float(), dim=-1)
output.argmax()

tensor(0)

In [5]:
from sklearn.metrics import accuracy_score, f1_score
from datasets import load_dataset
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd

def map_output(arg):
  dic = {0: 2, 1: 0}
  return dic[arg]


def test_sentiment140(model, tokenizer, batch_size=8):
    dataset = load_dataset('pauri32/fiqa-2018')
    dataset = dataset["test"]
    dataset = dataset.to_pandas()

    dataset["target"] = dataset['label']
    dataset = dataset[dataset["target"] != 1]

    dataset = dataset[['sentence', 'target']]
    dataset.columns = ['text', 'target']

    # print example
    print(f"\n\nPrompt example:\n{dataset['text'][1]}\n\n")

    context = dataset['text'].tolist()
    total_steps = dataset.shape[0]//batch_size + 1
    print(
        f"Total len: {len(context)}. Batchsize: {batch_size}. Total steps: {total_steps}")

    out_text = []

    for i in tqdm(range(total_steps)):
        tmp_context = context[i * batch_size:(i+1) * batch_size]

        tokenizer.pad_token = "[PAD]"

        tokens = tokenizer(tmp_context, return_tensors='pt',
                           padding=True)

        output = model(**tokens)
        output = torch.nn.functional.softmax(output.logits.float(), dim=-1)
        out_text.append(output.detach().numpy())
        torch.cuda.empty_cache()

    out_text = [item for sublist in out_text for item in sublist]
    dataset["out_text"] = out_text
    dataset["new_out"] = dataset["out_text"].apply(np.argmax).apply(map_output)

    acc = accuracy_score(dataset["target"], dataset["new_out"])
    f1_macro = f1_score(dataset["target"], dataset["new_out"], average="macro")
    f1_micro = f1_score(dataset["target"], dataset["new_out"], average="micro")
    f1_weighted = f1_score(
        dataset["target"], dataset["new_out"], average="weighted")

    print(f"Acc: {acc}. F1 macro: {f1_macro}. F1 micro: {f1_micro}. F1 weighted (BloombergGPT): {f1_weighted}. ")

    return dataset

dataset = test_sentiment140(model, tokenizer)
dataset



Prompt example:
@gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?


Total len: 100. Batchsize: 8. Total steps: 13


100%|██████████| 13/13 [00:00<00:00, 35.97it/s]

Acc: 0.63. F1 macro: 0.5960257670051317. F1 micro: 0.63. F1 weighted (BloombergGPT): 0.5960257670051317. 


,text,target,out_text,new_out
0,$HCP Come to the party and buy this -gonna giv...,0,"[0.4479206, 0.5520794]",0
1,@gakrum nice chart shows distinctive down chan...,2,"[0.980034, 0.019966036]",2
2,Japan's Asahi to submit bid next week for SABM...,0,"[0.9754836, 0.024516335]",2
3,"Tesla Motors recalls 2,700 Model X SUVs $TSLA ...",2,"[0.99649614, 0.003503823]",2
4,CRH's concrete bid for Holcim Lafarge assets,0,"[0.98498523, 0.015014747]",2
...,...,...,...,...
141,FDA Approves AstraZeneca's Iressa As Lung Canc...,0,"[0.8407172, 0.15928277]",2
142,energy still failing to close above the 10D MA...,2,"[0.999757, 0.00024298552]",2
143,AstraZeneca's patent on asthma drug invalidate...,2,"[0.9983026, 0.0016974094]",2
145,Intertek swings to ÃÂ£347 mln loss on oil's s...,2,"[0.99936503, 0.00063495064]",2


In [6]:
dataset["text"].iloc[0]

'$HCP Come to the party and buy this -gonna give solid gains and a dividend $$$$$$'

In [7]:
path = '../../results/financial/distilbert-base-uncased-finetuned-sst-2-english.csv'

In [8]:
dataset.to_csv(path)

In [9]:
import pandas as pd

df = pd.read_csv(path, index_col=0)

true, pred = df["target"], df["new_out"]


import sys
sys.path.append('../../')
from metrics import metrics

metrics(true, pred)

Precision: 0.696, Recall: 0.63, F1: 0.596, Accuracy: 0.63
